In [ ]:
# !pip uninstall -y numpy==1.23.1
# !pip install numpy==1.21.6
# !pip install --upgrade numpy
!pwd

In [ ]:
!nvidia-smi
import torch
import sys
sys.path.append('/home/apsisdev/mobassir/data/tts/bn_GlowTTS/TTS')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.

from TTS.tts.utils.text.tokenizer import TTSTokenizer
# BaseDatasetConfig: defines name, formatter and path of the dataset.


from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig


from TTS.config import load_config


from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

In [ ]:
# ## Install Coqui TTS
# ! pip install -U pip
# ! pip install TTS

# !git clone https://github.com/coqui-ai/TTS
# os.chdir('TTS/')
# !python3 -m pip install -r requirements.txt
# !python3 setup.py develop


!ls

In [ ]:
# # !apt install -y espeak
# !apt install -y espeak-ng
!pwd

In [ ]:

# output_path = "tts_train_dir"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

male = False
is_pretrain = False


meta_file = '/home/apsisdev/mobassir/data/tts/bn_tts/female/mono/metadata_female.txt'
root_path = '/home/apsisdev/mobassir/data/tts/bn_tts/female/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
    dataset_name="ljspeech", meta_file_train="/home/apsisdev/mobassir/data/tts/bn_tts/female/mono/metadata_female.txt", path=os.path.join("/home/apsisdev/mobassir/data/tts/bn_tts/female/mono/", "")
)


In [ ]:
dataset_config

In [ ]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

In [ ]:
print(len(train_samples),len(eval_samples))

In [ ]:
train_samples[0]

In [ ]:
# we use the same path as this script as our training folder.
output_path = '/home/apsisdev/mobassir/data/tts/'

audio_config = BaseAudioConfig(
     sample_rate = 22050,
    resample =False
)

if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ\u200cঁ৪৫ত",
    punctuations = ".?-!|, ",
    )

# GlowTTSConfig: all model related values for training, validating and testing.

config = GlowTTSConfig(
    #use_speaker_embedding = True,
    run_name="bangla_glowtts_female",
    batch_size=196,
    eval_batch_size=124,
    num_loader_workers=16,
    num_eval_loader_workers=16,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=15000,
    text_cleaner="collapse_whitespace",
    use_phonemes=False,
#     phoneme_language="bn",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=50,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    audio = audio_config,
    characters = characters_config,
    cudnn_benchmark=True,
    save_step=1000,
    test_sentences = [
#         "পিপলস ইন্স্যুরেন্স অব চায়না ছেষট্টি বছর আগে ব্যবসা চালু করে।",
#         "সোনার বাংলা আমি তোমায় ভালবাসি।"
        "হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।",
        "দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।",
        "হওয়ার,হয়েছে,নিশ্চয়ই,রায়,কিয়ামত,উভয়,দিয়েছেন,দুনিয়া,ন্যায়,অবস্থায়,যায়,ফিরিয়ে,দিয়েছিল,ভয়ে,দ্বিতীয়,দায়ক,পায়।",
        "গিয়ে,চেয়ে,হিদায়াত,দায়ে,নিয়েছ,রয়েছে,শয়তান,কিয়ামতে,সম্প্রদায়ে,সম্প্রদায়ের,নেয়,জয়,কিয়ামতের,স্থায়ী,যাওয়া,দয়ালু।",
        "ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।"
    ]
)

In [ ]:

ap = AudioProcessor.init_from_config(config)


In [ ]:
ap.resample

In [ ]:

tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:

model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [ ]:
model

In [ ]:
if(is_pretrain):
    pretrain_path =  '/home/apsisdev/mobassir/data/tts/bn_GlowTTS/female/'
else:
    pretrain_path = ''

In [ ]:
# using continue_path argument below for retraining model from last saved weight https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py#L210

trainer = Trainer(
    TrainerArgs(continue_path = pretrain_path), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

In [ ]:
%%time

# AND... 3,2,1... 🚀
trainer.fit()

In [ ]:
!nvidia-smi

In [ ]:
np.__version__#1.21.6